# Deep and Reinforcement Learning 2024/2025 (M.IA003), FEUP/FCUP

## Deep Learning Project 
## **Develop deep learning discriminative and generative models, applied to the context of “deep fakes”**

work done by:
- Michal Kowalski up202401554
- Pedro Pereira up201708807
- Pedro Azevedo up201905966

## 1.1) Import Necessary Files

In [165]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import glob
from _model import *
from tensorflow.keras.models import load_model
import json

# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [167]:
# Define dataset directories
data_dir = "data"
real_dir = os.path.join(data_dir, "real")
fake1_dir = os.path.join(data_dir, "fake1")
fake2_dir = os.path.join(data_dir, "fake2")
fake3_dir = os.path.join(data_dir, "fake3")

In [6]:
# data_dir = "data/celeba"
# real_dir = os.path.join(data_dir, "img_align_celeba")

# import tensorflow_datasets as tfds 

# celeba_builder = tfds.builder('celeb_a')
# celeba_builder.download_and_prepare(download_dir=data_dir)

## 1.2) Basic EDA

In [12]:
import os
import shutil
def move_images_to_main_folder(source_dir):
    """Move images from subdirectories to the main folder and remove empty subdirectories."""

    for subdir in os.listdir(source_dir):
        subdir_path = os.path.join(source_dir, subdir)

        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)

                if os.path.isfile(file_path):

                    target_path = os.path.join(source_dir, filename)

                    shutil.move(file_path, target_path)
                    print(f"Moved {filename} to {target_path}")


            if not os.listdir(subdir_path):
                os.rmdir(subdir_path)
                print(f"Removed empty subdirectory: {subdir_path}")

In [14]:
from _eda import *
base_stats(real_dir, fake1_dir, fake2_dir, fake3_dir)

Skipping .DS_Store: cannot identify image file '/Users/me/Desktop/Discriminative-Generative-Models-ACPR-/data/real/.DS_Store'

Real Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 28952, 'PNG': 1005, 'GIF': 36, 'MPO': 7})
Color Modes: Counter({'RGB': 26445, 'L': 2840, 'RGBA': 664, 'P': 40, 'LA': 11})
Top 5 Sizes: [((440, 660), 1469), ((440, 587), 959), ((440, 293), 688), ((440, 330), 579), ((440, 440), 272)]

Fake1 Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 30000})
Color Modes: Counter({'RGB': 30000})
Top 5 Sizes: [((512, 512), 30000)]

Fake2 Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 30000})
Color Modes: Counter({'RGB': 30000})
Top 5 Sizes: [((512, 512), 30000)]

Fake3 Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 30000})
Color Modes: Counter({'RGB': 30000})
Top 5 Sizes: [((512, 512), 30000)]


## 1.3) File Standardization 

In [129]:
import random
# Keep 15000 images from each fake images set to keep the balance with real images set
def select_and_remove_images(source_dir, num_images_to_keep=20000):
    all_images = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    if len(all_images) <= num_images_to_keep:
        print(f"Not enough images to remove. Keeping {len(all_images)} images.")
        return
    random.shuffle(all_images)
    images_to_remove = all_images[num_images_to_keep:]
    for image in images_to_remove:
        os.remove(image)
    print(f"Removed {len(images_to_remove)} images.")
    
source_dir1 = "data/fake1_normalized_128"
source_dir2 = "data/fake2_normalized_128"
source_dir3 = "data/fake3_normalized_128"
select_and_remove_images(source_dir1, num_images_to_keep=15000)
select_and_remove_images(source_dir2, num_images_to_keep=15000)
select_and_remove_images(source_dir3, num_images_to_keep=15000)

Removed 5000 images.
Removed 5000 images.
Removed 5000 images.


In [181]:
# from _preprocess import standardize_files
# Standarize real images
standardize_files('data/real')

data/real/35079884_1994-08-07_2012.jpg
data/real/4478357_1945-01-12_1972.jpg
data/real/35336978_1990-11-28_2011.jpg
data/real/12999768_1977-03-24_2000.jpg
data/real/100936_1944-05-25_2012.jpg
data/real/46397697_1974-03-25_2005.jpg
data/real/24796560_1968-01-08_2009.jpg
data/real/36118999_1983-08-12_2011.jpg
data/real/31160609_1973-08-18_2014.jpg
data/real/30805326_1991-01-18_2012.jpg
data/real/5059042_1986-03-17_2007.jpg
data/real/201525_1956-10-17_1992.jpg
data/real/17093549_1976-09-11_2009.jpg
data/real/24252679_1986-02-26_2013.jpg
data/real/2500405_1954-02-26_2010.jpg
data/real/5134483_1980-04-05_2012.jpg
data/real/41424733_1945-06-03_1973.jpg
data/real/33073820_1975-08-31_2011.jpg
data/real/7357887_1989-08-27_2007.jpg
data/real/327158_1949-02-17_1998.jpg
data/real/11895475_1987-07-27_2015.jpg
data/real/1891926_1947-07-05_2012.jpg
data/real/978073_1923-01-30_1953.jpg
data/real/36963786_1977-01-04_2012.jpg
data/real/7449432_1947-05-18_2007.jpg
data/real/43063338_1991-01-26_2014.jpg
d

In [186]:
from _eda import *
real_dir = os.path.join(data_dir, "real_normalized")
fake1_dir = os.path.join(data_dir, "fake1")
fake2_dir = os.path.join(data_dir, "fake2")
fake3_dir = os.path.join(data_dir, "fake3")
base_stats(real_dir, fake1_dir, fake2_dir, fake3_dir)


Real Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 30000})
Color Modes: Counter({'RGB': 30000})
Top 5 Sizes: [((440, 660), 1469), ((440, 587), 959), ((440, 293), 688), ((440, 330), 579), ((440, 440), 272)]

Fake1 Images Summary:
Total images: 15000
Formats: Counter({'JPEG': 15000})
Color Modes: Counter({'RGB': 15000})
Top 5 Sizes: [((512, 512), 15000)]

Fake2 Images Summary:
Total images: 15000
Formats: Counter({'JPEG': 15000})
Color Modes: Counter({'RGB': 15000})
Top 5 Sizes: [((512, 512), 15000)]

Fake3 Images Summary:
Total images: 15000
Formats: Counter({'JPEG': 15000})
Color Modes: Counter({'RGB': 15000})
Top 5 Sizes: [((512, 512), 15000)]


# 1.5) Train, Validation, Test Dataset Split

In [218]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

def create_train_val_test_split(real_dirs, fake_dirs, output_dir, test_val_size_real=0.15, test_val_size_fake=0.15):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train/real'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train/fake'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val/real'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val/fake'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test/real'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test/fake'), exist_ok=True)

    # Collect real images
    real_images = []
    for real_dir in real_dirs:
        for real_dir_item in os.listdir(real_dir):
            real_images.append(os.path.join(real_dir, real_dir_item))

    fake_images = []
    for fake_dir in fake_dirs:
        for fake_dir_item in os.listdir(fake_dir):
            fake_images.append(os.path.join(fake_dir, fake_dir_item))


    # Shuffle real and fake images
    random.shuffle(real_images)
    random.shuffle(fake_images)

    # # Perform the train/test split
    real_train, real_temp = train_test_split(real_images, test_size=test_val_size_real * 2, random_state=42)
    real_val, real_test = train_test_split(real_temp, test_size=0.5, random_state=42)

    fake_train, fake_temp = train_test_split(fake_images, test_size=test_val_size_fake * 2, random_state=42)
    fake_val, fake_test = train_test_split(fake_temp, test_size=0.5, random_state=42)

    print('Real Train: ', len(real_train))
    print('Real Val: ', len(real_val))
    print('Real Test: ', len(real_test))

    print('Fake Train: ', len(fake_train))
    print('Fake Val: ', len(fake_val))
    print('Fake Test: ', len(fake_test))

    return real_train, real_val, real_test, fake_train, fake_val, fake_test

In [220]:
real_dirs = ['data/real_normalized']
fake_dirs = ['data/fake1', 'data/fake2', 'data/fake3']
output_dir = 'data/split'

real_train, real_val, real_test, fake_train, fake_val, fake_test = create_train_val_test_split(real_dirs, fake_dirs, output_dir, test_val_size_real=0.225, test_val_size_fake=0.15)

Real Train:  16500
Real Val:  6750
Real Test:  6750
Fake Train:  31500
Fake Val:  6750
Fake Test:  6750


In [222]:
import os
import shutil

# Function to copy images with a unique name (if the file already exists, it adds a number)
def copy_image_with_unique_name(src, dest_dir):
    file_name = os.path.basename(src)
    dest_path = os.path.join(dest_dir, file_name)
    base_name, ext = os.path.splitext(file_name)
    counter = 1

    while os.path.exists(dest_path):
        dest_path = os.path.join(dest_dir, f"{base_name}_{counter}{ext}")
        counter += 1

    shutil.copy(src, dest_path)

# Function to copy images to the correct directories (train, val, test)
def copy_images_to_dirs(images, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for img in images:
        copy_image_with_unique_name(img, dest_dir)

# Function to copy split data (train, val, test for real and fake images)
def copy_split_data(real_train, real_val, real_test, fake_train, fake_val, fake_test, output_dir):
    # Dictionaries structure
    os.makedirs(os.path.join(output_dir, 'train/real'), exist_ok=True)
    # os.makedirs(os.path.join(output_dir, 'train/fake'), exist_ok=True)
    # os.makedirs(os.path.join(output_dir, 'val/real'), exist_ok=True)
    # os.makedirs(os.path.join(output_dir, 'val/fake'), exist_ok=True)
    # os.makedirs(os.path.join(output_dir, 'test/real'), exist_ok=True)
    # os.makedirs(os.path.join(output_dir, 'test/fake'), exist_ok=True)

    # Copying images to the directories
    copy_images_to_dirs(real_train, os.path.join(output_dir, 'train/real'))
    # copy_images_to_dirs(real_val, os.path.join(output_dir, 'val/real'))
    # copy_images_to_dirs(real_test, os.path.join(output_dir, 'test/real'))

    # copy_images_to_dirs(fake_train, os.path.join(output_dir, 'train/fake'))
    # copy_images_to_dirs(fake_val, os.path.join(output_dir, 'val/fake'))
    # copy_images_to_dirs(fake_test, os.path.join(output_dir, 'test/fake'))

In [224]:
output_dir = 'data/split'
# Call the function to copy train, val, test sets into the correct dictionaries
copy_split_data(real_train, real_val, real_test, fake_train, fake_val, fake_test, output_dir)

In [226]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import random

input_folder = 'data/split/train/real'
output_folder = 'data/split/train/real'

# Create folder for augmented images if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get random images for the augmentation
def get_random_filenames(filenames, num_images):
    random.shuffle(filenames)  # Shuffle filenames to ensure randomness
    return filenames[:num_images]

# Flip horizontally and adjust hue
def flip_hue_augmentation(input_folder, output_folder, filenames, num_augmented_images=5):
    augmented_count = 0
    random_filenames = get_random_filenames(filenames, num_augmented_images)

    for filename in random_filenames:
        img_path = os.path.join(input_folder, filename)

        if os.path.isfile(img_path):
            img = image.load_img(img_path)
            img_array = image.img_to_array(img)

            flipped_image = tf.image.flip_left_right(img_array)
            hue_adjusted_image = tf.image.adjust_hue(flipped_image, delta=0.1)
            hue_adjusted_image = np.array(hue_adjusted_image, dtype=np.uint8)
            hue_pil_image = Image.fromarray(hue_adjusted_image)

            new_filename = f'{os.path.splitext(filename)[0]}_fh.jpg'
            hue_pil_image.save(os.path.join(output_folder, new_filename))

            augmented_count += 1

    print(f"Horizontal flip and hue adjustment complete, {augmented_count} images saved.")

# Change brightness and hue 
def brightness_hue_augmentation(input_folder, output_folder, filenames, num_augmented_images=5, brightness_delta=0.2, hue_delta=0.05):
    augmented_count = 0
    random_filenames = get_random_filenames(filenames, num_augmented_images)

    for filename in random_filenames:
        img_path = os.path.join(input_folder, filename)

        if os.path.isfile(img_path):
            img = image.load_img(img_path)
            img_array = image.img_to_array(img)

            if len(img_array.shape) == 3 and img_array.shape[2] == 1:
                img_array = np.repeat(img_array, 3, axis=-1)

            brightened_image = tf.image.adjust_brightness(img_array, delta=brightness_delta)
            hue_adjusted_image = tf.image.adjust_hue(brightened_image, delta=hue_delta)

            hue_brightness_adjusted_image = np.array(hue_adjusted_image, dtype=np.uint8)
            hue_brightness_pil_image = Image.fromarray(hue_brightness_adjusted_image)

            new_filename = f'{os.path.splitext(filename)[0]}_bh.jpg'
            hue_brightness_pil_image.save(os.path.join(output_folder, new_filename))

            augmented_count += 1

    print(f"Brightness and hue adjustment complete, {augmented_count} images saved.")



# Zoom the image
def zoom_augmentation(input_folder, output_folder, filenames, num_augmented_images=5, zoom_factor=0.2):
    augmented_count = 0
    random_filenames = get_random_filenames(filenames, num_augmented_images)

    for filename in random_filenames:
        img_path = os.path.join(input_folder, filename)

        if os.path.isfile(img_path):
            img = image.load_img(img_path)
            img_array = image.img_to_array(img)

            height, width, _ = img_array.shape
            zoomed_image = tf.image.resize(img_array, [int(height * (1 + zoom_factor)), int(width * (1 + zoom_factor))])
            cropped_image = tf.image.resize_with_crop_or_pad(zoomed_image, target_height=height, target_width=width)
            cropped_image = np.array(cropped_image, dtype=np.uint8)
            zoomed_pil_image = Image.fromarray(cropped_image)

            # Add 'd' suffix to avoid overwriting
            new_filename = f'{os.path.splitext(filename)[0]}_z.jpg'
            zoomed_pil_image.save(os.path.join(output_folder, new_filename))

            augmented_count += 1

    print(f"Zoom augmentation complete, {augmented_count} images saved.")

# Get the list of filenames (image names) in the input folder
filenames = os.listdir(input_folder)
# Perform augmentations with suffixes
brightness_hue_augmentation(input_folder, output_folder, filenames, num_augmented_images=5000)
flip_hue_augmentation(input_folder, output_folder, filenames, num_augmented_images=5000)
zoom_augmentation(input_folder, output_folder, filenames, num_augmented_images=5000, zoom_factor=0.3)

Brightness and hue adjustment complete, 5000 images saved.
Horizontal flip and hue adjustment complete, 5000 images saved.
Zoom augmentation complete, 5000 images saved.


## 1.6) Modeling

In [228]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
# Normalizing pixels values to [0,1]
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [234]:
# Paths to datasets
train_dir = 'data/split/train'
val_dir = 'data/split/val'
test_dir = 'data/split/test'
# Load datasets and split data into batches 

# Example batch size 64
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary',# Binary classification (real vs fake)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary',
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary',
)

Found 63000 images belonging to 2 classes.
Found 13500 images belonging to 2 classes.
Found 13500 images belonging to 2 classes.


In [ ]:
# Train/Val/Test: 70/15/15

In [232]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback after 5 iterations without improvement
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=3,
                               restore_best_weights=True)

In [236]:
from tensorflow.keras import layers, models

# the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.BatchNormalization(),  # Dodanie normalizacji wsadowej
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),  # Dodanie normalizacji wsadowej
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
# Model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_110 (Conv2D)         (None, 198, 198, 32)      896       
                                                                 
 batch_normalization_94 (Bat  (None, 198, 198, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 99, 99, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_111 (Conv2D)         (None, 97, 97, 64)        18496     
                                                                 
 batch_normalization_95 (Bat  (None, 97, 97, 64)       256       
 chNormalization)                                                
                                                      

In [238]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall']
)

In [113]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
)

Epoch 1/15


2025-04-02 05:19:00.654307: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1312/1312 [==============================] - ETA: 0s - loss: 0.6479 - accuracy: 0.6135 - precision: 0.5974 - recall: 0.6955

2025-04-02 05:43:46.165898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1312/1312 [==============================] - 1576s 1s/step - loss: 0.6479 - accuracy: 0.6135 - precision: 0.5974 - recall: 0.6955 - val_loss: 0.5974 - val_accuracy: 0.6531 - val_precision: 0.6162 - val_recall: 0.8122
Epoch 2/15
1312/1312 [==============================] - 1358s 1s/step - loss: 0.5825 - accuracy: 0.6608 - precision: 0.6367 - recall: 0.7490 - val_loss: 0.5681 - val_accuracy: 0.6659 - val_precision: 0.6366 - val_recall: 0.7725
Epoch 3/15
1312/1312 [==============================] - 1363s 1s/step - loss: 0.5587 - accuracy: 0.6783 - precision: 0.6499 - recall: 0.7733 - val_loss: 0.5809 - val_accuracy: 0.6574 - val_precision: 0.6125 - val_recall: 0.8565
Epoch 4/15
1312/1312 [==============================] - 1389s 1s/step - loss: 0.5348 - accuracy: 0.6965 - precision: 0.6679 - recall: 0.7815 - val_loss: 0.5908 - val_accuracy: 0.6602 - val_precision: 0.6439 - val_recall: 0.7167
Epoch 5/15
1312/1312 [==============================] - 1427s 1s/step - loss: 0.5064 - accuracy: 0.

## 1.4) Baseline

In [ ]:
gan = build_gan(label_smoothing=True, wgan=True)

In [ ]:
gan.generator.summary()

In [ ]:

batch_size = 256
# Get all image file paths

def save_paths(path_list, fname):
    with open(fname, "w") as final:
	    json.dump(path_list, final)

# Define the preprocessing function
def load_and_preprocess(image_path):
    # Read the image file
    image = tf.io.read_file(image_path)

    # Decode the JPEG image
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize the image to the range [-1, 1]
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0

    return image

def load_paths(train_file, test_file):
    with open(train_file, "r") as train:
	    train_paths = json.load(train)
    with open(test_file, "r") as test:
	    test_paths = json.load(test)
    return train_paths, test_paths

In [ ]:
# image_paths = glob.glob("./data/celeba/img_align_celeba_normalized_64/*.jpg")
# train_paths = image_paths[:150000]
# test_paths = image_paths[150000:]
# save_paths(train_paths, 'train.json')
# save_paths(test_paths, 'test.json')

image_paths = glob.glob("./data/real_normalized_64/*.jpg")

#train_paths, test_paths = load_paths('./train.json', './test.json')

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices(train_paths)
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
gan.discriminator.summary()

In [ ]:
d_losses, e_losses = train_gan(gan, dataset, epochs=40, batch_size=batch_size)

In [ ]:
gan.save('celeba_wgan.keras')

In [ ]:
final_d_losses = [float(x) for x in d_losses]
final_g_losses = [float(x) for x in e_losses]
with open('d_losses.json', "w") as f:
    json.dump(final_d_losses, f)

with open('g_losses.json', "w") as f:
    json.dump(final_g_losses, f)

# 1.5) Load Model

In [ ]:
gan = keras.models.load_model("celeba_wgan.keras", custom_objects={"GAN": GAN})

In [ ]:
gan.summary()

In [ ]:
gan.generator.summary()

In [ ]:
def generate_image(generator):
    noise = np.random.normal(0, 1, (1, LATENT_DIM))
    gen_images = generator.predict(noise)
    gen_images = (gen_images + 1) / 2  # Rescale images to [0,1]
    plt.imshow(gen_images[0])
    plt.show()

generate_image(gan.generator)


# 1.6) Test Generator

In [ ]:
insight_paths = glob.glob("./data/fake_normalized_64/*.jpg")
inpainting_paths = glob.glob("./data/inpainting_normalized_64/*.jpg")
text2img_paths = glob.glob("./data/text2img_normalized_64/*.jpg")

_, real_paths = load_paths('./train.json', './test.json')

# Create the dataset
dataset_real = tf.data.Dataset.from_tensor_slices(real_paths)
dataset_real = dataset_real.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset_real = dataset_real.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

dataset_insight = tf.data.Dataset.from_tensor_slices(insight_paths)
dataset_insight = dataset_insight.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset_insight = dataset_insight.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

dataset_inpainting = tf.data.Dataset.from_tensor_slices(inpainting_paths)
dataset_inpainting = dataset_inpainting.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset_inpainting = dataset_inpainting.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

dataset_text2img = tf.data.Dataset.from_tensor_slices(text2img_paths)
dataset_text2img = dataset_text2img.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset_text2img = dataset_text2img.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
from _fid import *

def gen_experiments(dataset_real, dataset_insight, dataset_inpainting, dataset_text2img, batch_size, num_batches=10):
    gen_fids = []
    insight_fids = []
    inpainting_fids = []
    text2img_fids = []
    for i in range(num_batches):

        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        gen_images = gan.generator.predict(noise)

        real_images = next(iter(dataset_real.take(1)))[0].numpy()
        real_images = real_images.astype(np.float32)

        insight_images = next(iter(dataset_insight.take(1)))[0].numpy()
        insight_images = insight_images.astype(np.float32)

        inpainting_images = next(iter(dataset_inpainting.take(1)))[0].numpy()
        inpainting_images = inpainting_images.astype(np.float32)

        text2img_images = next(iter(dataset_text2img.take(1)))[0].numpy()
        text2img_images = text2img_images.astype(np.float32)

        gen_images = img_scaler(gen_images, (75,75,3))
        real_images = img_scaler(real_images, (75,75,3))
        insight_images = img_scaler(insight_images, (75, 75, 3))
        inpainting_images = img_scaler(inpainting_images, (75, 75, 3))
        text2img_images = img_scaler(text2img_images, (75, 75, 3))

        gen_fid = calculate_fid(inception_model, gen_images, real_images)
        gen_fids.append(gen_fid)

        insight_fid = calculate_fid(inception_model, insight_images, real_images)
        insight_fids.append(insight_fid)

        inpainting_fid = calculate_fid(inception_model, inpainting_images, real_images)
        inpainting_fids.append(inpainting_fid)

        text2img_fid = calculate_fid(inception_model, text2img_images, real_images)
        text2img_fids.append(text2img_fid)

    return np.mean(gen_fids), gen_fids, np.mean(insight_fids), insight_fids, np.mean(inpainting_fids), inpainting_fids, np.mean(text2img_fids), text2img_fids

mean_gen, gen_fids, mean_ins, ins_fids, mean_inp, inp_fids, mean_t2i, t2i_fids = gen_experiments(dataset_real, dataset_insight, dataset_inpainting, dataset_text2img, batch_size//2, num_batches=1)

In [ ]:
mean_t2i